In [1]:
import time
from datetime import date
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

In [2]:
url = "https://kind.krx.co.kr/corpgeneral/corpList.do"
payload = dict(method='searchCorpList',pageIndex=1,currentPageSize=15,comAbbrv="",beginIndex="",orderMode='3',orderStat='D',isurCd="",repIsuSrtCd="",searchCodeType="",marketType="",searchType='13',industry="",fiscalYearEnd='all',comAbbrvTmp="",location='all')
r = requests.get(url, params = payload)
print(r.url)
soup = bs(r.text, 'lxml')

https://kind.krx.co.kr/corpgeneral/corpList.do?method=searchCorpList&pageIndex=1&currentPageSize=15&comAbbrv=&beginIndex=&orderMode=3&orderStat=D&isurCd=&repIsuSrtCd=&searchCodeType=&marketType=&searchType=13&industry=&fiscalYearEnd=all&comAbbrvTmp=&location=all


In [3]:
keys = soup.select_one("table.list.type-00.tmt30")['summary'].split(", ")

In [4]:
page = 1
page_size = 100
final_result_df = pd.DataFrame()
while True:
    url = "https://kind.krx.co.kr/corpgeneral/corpList.do"
    payload = dict(method='searchCorpList',pageIndex=page, currentPageSize=page_size,orderMode=3,orderStat='D',searchType=13, fiscalYearEnd='all', location='all')
    r = requests.get(url, params=payload)
    print(r.status_code, end="\r")
    soup = bs(r.text, 'lxml')
    total_items = int(soup.select_one(".info.type-00 > em").text.replace(",", ""))
    total_pages = total_items // page_size + 1
    print(f"{page}/{total_pages} 수집중", end="\r")
    keys = soup.select_one("table.list.type-00.tmt30")['summary'].split(", ")  
    result = {}
    for tr in soup.select('tr'):
        for idx, (key, td) in enumerate(zip(keys, tr.select('td'))):
            if idx == 0:
                kinds = [img['alt'].strip() for img in td.select('img')]   # 1번째 증권 종류, 회사이름
                kind = ", ".join(kinds)
                code = td.select_one("a")['onclick'].split("'")[1] # 종목코드 추출
                result.setdefault('증권종류', []).append(kind)
                result.setdefault(key, []).append(td.text)
                result.setdefault('종목코드', []).append(code+"0") # 종목코드 추가
            elif idx == 6:
                home_link = td.select_one('a')['href'] if td.string == None else ""  # 6번째 링크 찾기
                result.setdefault(key, []).append(home_link)
            else:
                result.setdefault(key, []).append(td.text)
    result_df = pd.DataFrame(result)
    final_result_df = pd.concat([final_result_df, result_df])
        
    if page < total_pages:
        page += 1
        time.sleep(5)
    else:
        break

display(final_result_df)

,증권종류,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,코스닥,미래에셋비전스팩7호,482680,금융 지원 서비스업,기업인수합병,2024-09-11,12월,송우영,,서울특별시
1,코스닥,아이비젼웍스,469750,특수 목적용 기계 제조업,이차전지 검사시스템 및 검사시스템 설치셋업용역,2024-09-03,12월,길기재,http://www.ivw.co.kr/kor/main/,대전광역시
2,코스닥,아이스크림미디어,461300,소프트웨어 개발 및 공급업,"교육출판, 커머스, 연수",2024-08-30,12월,허주환,http://www.i-screammedia.com,경기도
3,"코스닥, 투자경고종목",이엔셀,456070,기초 의약물질 제조업,"첨단바이오의약품 위탁개발생산(CDMO) 서비스 / 차세대 세포,유전자 치료제",2024-08-23,12월,장종욱,http://www.encellinc.com/,서울특별시
4,코스닥,M83,476080,"영화, 비디오물, 방송프로그램 제작 및 배급업",VFX,2024-08-22,12월,정성진..,http://m83.co.kr,서울특별시
...,...,...,...,...,...,...,...,...,...,...
11,"유가증권, KTOP30, KOSPI200, KRX300",유한양행,000100,의약품 제조업,"의약품(삐콤씨, 안티푸라민, 렉라자, 로수바미브, 코푸시럽 등), 생활용품(유한락스...",1962-11-01,12월,대표이..,http://www.yuhan.co.kr,서울특별시
12,"유가증권, KOSPI200, KRX300",CJ대한통운,000120,도로 화물 운송업,"Contract Logistics, 포워딩, 항만하역, 해운, 택배국제특송, SCM...",1956-07-02,12월,신영수..,http://www.cjlogistics.com,서울특별시
13,유가증권,경방,000050,종합 소매업,"섬유류(면사,면혼방사,면직물,면혼방직물,화섬사,화섬직물) 제조,도매,수출입",1956-03-03,12월,"김준,..",http://www.kyungbang.co.kr,서울특별시
14,유가증권,유수홀딩스,000700,회사 본부 및 경영 컨설팅 서비스업,지주사업,1956-03-03,12월,송영규,http://www.eusu-holdings.com,서울특별시


# DB에 상장기업정보 저장하기

In [5]:
pip install cryptography

Note: you may need to restart the kernel to use updated packages.


In [9]:
import os
from datetime import datetime
from sqlalchemy import create_engine, text
import pymysql
pymysql.install_as_MySQLdb()

# 데이터베이스 접속 정보 설정 (환경 변수 대신 문자열을 직접 사용)
db = "mysql"
dbtype = "pymysql"
user = "root"
password = "1234"
host = "127.0.0.1:3306"
database = "korean_stock"

# MySQL 서버에 연결 (데이터베이스를 지정하지 않고 접속)
engine = create_engine(f"{db}+{dbtype}://{user}:{password}@{host}")

conn = engine.connect()

# 데이터베이스 삭제 및 생성
conn.execute(text(f"DROP DATABASE IF EXISTS {database}"))
conn.execute(text(f"CREATE DATABASE {database}"))
conn.close()

# 새로 생성한 데이터베이스로 다시 연결
engine = create_engine(f"{db}+{dbtype}://{user}:{password}@{host}/{database}")
conn = engine.connect()

# DataFrame을 MySQL 테이블로 저장 (final_result_df가 정의되어 있어야 함)
# final_result_df는 사용자가 사전에 정의한 DataFrame이어야 합니다.
final_result_df.to_sql('stock_company_info_김지연', con=conn, if_exists='append', index=False)
conn.close()

In [11]:
pip install pyinstaller

Note: you may need to restart the kernel to use updated packages.


In [12]:
pyinstaller -F stock_company_info.ipynb

SyntaxError: invalid syntax (4026774543.py, line 1)